In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import string
import nltk
nltk.download('stopwords')
import tensorflow as tf
from keras.callbacks import EarlyStopping



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mohammed/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2023-08-11 08:05:18.147931: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-11 08:05:18.644712: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-11 08:05:18.647069: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-11 08:05:20.248950: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data = pd.read_csv('./data/Stress.csv')
data.head()

,subreddit,post_id,sentence_range,text,label,confidence,social_timestamp
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",1,0.8,1521614353
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",0,1.0,1527009817
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,1,0.8,1535935605
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",1,0.6,1516429555
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1,0.8,1539809005


In [3]:
x = data['text'].astype(str)
y = data['label']

In [4]:
from sklearn.model_selection import train_test_split


In [5]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=10)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

In [7]:
X_train = X_train.astype(str).str.lower()
X_test = X_test.astype(str).str.lower()

In [8]:
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [10]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', LogisticRegression())
])

pipeline.fit(X_train, y_train)

accuracy = pipeline.score(X_test, y_test)

accuracy

0.7658450704225352

In [11]:
pipeline.score(X_test,y_test)

0.7658450704225352

In [12]:
sample_text = ["I feel stress because of having lot's of assignments in the collage and I have no time and my gpa is lower then 2"]

prediction = pipeline.predict(sample_text)

prediction_label = "Stressed" if prediction[0] == 1 else "Not Stressed"

prediction_label


'Stressed'

In [13]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score


In [14]:
y_pred = pipeline.predict(X_test)

In [15]:
precision = precision_score(y_test, y_pred)
precision

0.746875

In [16]:
recall = recall_score(y_test, y_pred)
recall

0.8213058419243986

In [17]:
# Initializing the TfidfVectorizer
tfidf = TfidfVectorizer()

# Transforming the 'text' column using the vectorizer
x = tfidf.fit_transform(data['text'])

# Getting the 'label' column values
y = data['label'].values

# Checking the shapes of the transformed data
x.shape, y.shape


((2838, 11516), (2838,))

In [18]:
X_train, X_test, y_train, y_test = train_test_split(x,y , test_size= 0.2, random_state= 40)
early_stopper = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [22]:
nn = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(1000, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(500, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(250, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(125, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

2023-08-11 08:06:25.453533: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-08-11 08:06:25.634846: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 46064000 exceeds 10% of free system memory.
2023-08-11 08:06:25.680751: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 46064000 exceeds 10% of free system memory.
2023-08-11 08:06:25.693275: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 46064000 exceeds 10% of free system memory.


In [23]:
precision_metric = tf.keras.metrics.Precision()
nn.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy', precision_metric])

In [24]:
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1000)              11517000  
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 500)               500500    
                                                                 
 dropout_1 (Dropout)         (None, 500)               0         
                                                                 
 dense_2 (Dense)             (None, 250)               125250    
                                                                 
 dropout_2 (Dropout)         (None, 250)               0         
                                                                 
 dense_3 (Dense)             (None, 125)               3

In [25]:
nn.fit(X_train.toarray(), y_train, epochs=100, batch_size=8, validation_data=(X_test.toarray(), y_test), callbacks=[early_stopper])


2023-08-11 08:06:50.143040: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 104565280 exceeds 10% of free system memory.


Epoch 1/100


2023-08-11 08:06:50.838432: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 46064000 exceeds 10% of free system memory.


284/284 [==============================] - 32s 106ms/step - loss: 0.6965 - accuracy: 0.5194 - precision: 0.5199 - val_loss: 0.6927 - val_accuracy: 0.5405 - val_precision: 0.5405
Epoch 2/100
284/284 [==============================] - 30s 107ms/step - loss: 0.6957 - accuracy: 0.5154 - precision: 0.5189 - val_loss: 0.6926 - val_accuracy: 0.5405 - val_precision: 0.5405
Epoch 3/100
284/284 [==============================] - 30s 107ms/step - loss: 0.6949 - accuracy: 0.5203 - precision: 0.5203 - val_loss: 0.6922 - val_accuracy: 0.5405 - val_precision: 0.5405
Epoch 4/100
284/284 [==============================] - 28s 99ms/step - loss: 0.6964 - accuracy: 0.5203 - precision: 0.5203 - val_loss: 0.6923 - val_accuracy: 0.5405 - val_precision: 0.5405
Epoch 5/100
284/284 [==============================] - 29s 103ms/step - loss: 0.6946 - accuracy: 0.5203 - precision: 0.5203 - val_loss: 0.6922 - val_accuracy: 0.5405 - val_precision: 0.5405
Epoch 6/100
284/284 [==============================] - 28s 98ms

In [26]:
reasults = nn.evaluate(X_test.toarray(), y_test)


18/18 [==============================] - 1s 13ms/step - loss: 0.6160 - accuracy: 0.7130 - precision: 0.7812


In [27]:
print(f"Test Loss: {reasults[0]:.4f}")
print(f"Test Accuracy: {reasults[1]:.4f}")

Test Loss: 0.6160
Test Accuracy: 0.7130


In [33]:
sample_text = ["I feel so much stress because of having lots of assignments in the collage and I have no time and my gpa is lower then 2"]
sample_text_tfidf = tfidf.transform(sample_text)
prediction = nn.predict(sample_text_tfidf.toarray())
prediction[0][0]

1/1 [==============================] - 0s 88ms/step


0.47340965

In [35]:
import os
checkpoint_path = "training/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [36]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [38]:
nn.fit(X_train.toarray(), y_train, epochs=10, batch_size=64, validation_data=(X_test.toarray(), y_test), callbacks=[early_stopper, cp_callback])

Epoch 1/10
36/36 [==============================] - ETA: 0s - loss: 0.6757 - accuracy: 0.5784 - precision: 0.8916
Epoch 1: saving model to training/cp.ckpt
36/36 [==============================] - 4s 103ms/step - loss: 0.6757 - accuracy: 0.5784 - precision: 0.8916 - val_loss: 0.6333 - val_accuracy: 0.6602 - val_precision: 0.8393
Epoch 2/10
36/36 [==============================] - ETA: 0s - loss: 0.6622 - accuracy: 0.5780 - precision: 0.9084
Epoch 2: saving model to training/cp.ckpt
36/36 [==============================] - 4s 102ms/step - loss: 0.6622 - accuracy: 0.5780 - precision: 0.9084 - val_loss: 0.6366 - val_accuracy: 0.6549 - val_precision: 0.8447
Epoch 3/10
36/36 [==============================] - ETA: 0s - loss: 0.6720 - accuracy: 0.5819 - precision: 0.9028
Epoch 3: saving model to training/cp.ckpt
36/36 [==============================] - 4s 104ms/step - loss: 0.6720 - accuracy: 0.5819 - precision: 0.9028 - val_loss: 0.6280 - val_accuracy: 0.6637 - val_precision: 0.8152
Epoch 4

In [39]:
os.listdir(checkpoint_dir)


['checkpoint', 'cp.ckpt.index', 'cp.ckpt.data-00000-of-00001']

In [40]:
tf.keras.models.save_model(nn, "training/stress.h5")

/tmp/ipykernel_27733/2146662781.py:1: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(nn, "training/stress.h5")
